In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [3]:
from typing import KeysView
import pandas as pd
import numpy as np
from collections import defaultdict, OrderedDict
import matplotlib.pyplot as plt

In [ ]:
'''
        cols=pd.Series(self.struct1.columns)
        for dup in self.struct1.columns[self.struct1.columns.duplicated(keep=False)]: 
          cols[self.struct1.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 else dup 
                                     for d_idx in range(self.struct1.columns.get_loc(dup).sum())]
                                    )
        self.struct1.columns=cols
        
        cols=pd.Series(self.struct2.columns)
        for dup in self.struct2.columns[self.struct2.columns.duplicated(keep=False)]: 
          cols[self.struct2.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 else dup 
                                     for d_idx in range(self.struct2.columns.get_loc(dup).sum())]
                                    )
        self.struct2.columns=cols
'''

In [ ]:
def typeChange(df):
  ranks = [r for r in df.columns if r.startswith("Rank")]
  for r in ranks:
    df[r] = df[r].astype(float)
  nMs = [n for n in df.columns if n.startswith("nM")]
  for n in nMs:
    df[n] = df[n].astype(float)
  haRanks = [h for h in df.columns if h.startswith("H_Avg_Ranks")]
  for h in haRanks:
    df[h] = df[h].astype(float)
  nBinders = [b for b in df.columns if b.startswith("N_binders")]
  for b in nBinders:
    df[b] = df[b].astype(int)
  return df

In [4]:
class NetMHC_Get_Dict:
    def __init__(self, file):
        self.df = pd.read_csv(file)
        self.hla_names = self.df.columns[self.df.columns.str.startswith("HLA")].tolist()
        self.df.columns = self.df.loc[0]
        self.df = self.df.iloc[1:, :]
        self.pos_id = self.df.iloc[:, :3]
        self.n_binders = self.df.iloc[:, -2:]
        self.df = self.df.iloc[:, 3:-2]

        self.hla_dict = {}
        
        for i, col_index in enumerate(range(0, len(self.df.columns), 3)):
            self.hla_dict[self.hla_names[i]] = pd.concat([self.pos_id,
                                                          self.df.iloc[:, col_index:col_index+3],
                                                          self.n_binders], axis=1)

    def get_dict(self):
        return self.hla_dict
    
class NetMHC_Comparative_Analysis:
    def __init__(self, file1, file2):
        self.struct1 = NetMHC_Get_Dict(file1).get_dict()
        self.struct2 = NetMHC_Get_Dict(file2).get_dict()
        
    def compare_top_binders(self):
        comp_dict, freq_epitopes = {}, defaultdict(int)
        
        for k, v in self.struct1.items():# keys were initially list of alleles
          self.struct1[k]["Rank"] = self.struct1[k]["Rank"].astype(float)
          self.struct2[k]["Rank"] = self.struct2[k]["Rank"].astype(float)
          struct1_top = self.struct1[k][self.struct1[k]["Rank"] < 0.5]
          struct2_top = self.struct2[k][self.struct2[k]["Rank"] < 0.5]            
          dif = struct1_top[~(struct1_top["Peptide"].isin(struct2_top["Peptide"]))]#compare two series elmtwise
          #ABOVE code returns unshared epitopes
          #print(dif)
          comp_dict[k] = (dif, len(dif))

          for pos, pep in zip(dif["Pos"], dif["Peptide"]):
            freq_epitopes[(pos, pep)] += 1
          
        freq_epitopes = {k:v for k, v in freq_epitopes.items()}
        #freq_diff_epitopes = dict(sorted(freq_epitopes.items(), key=lambda item: item[0]))

        #freq_diff_epitopes = pd.DataFrame(freq_epitopes.items(),
        #                                  columns=["Position and Peptide", "Count"])
        
        #freq_epitopes.plot(x="Position and Peptide", y="Count", kind="bar")
        #plt.title("Count of different epitopes - seen in first but not second protein")

        ####comp_dict_num_bind = pd.DataFrame({k:v[1] for k, v in comp_dict.items()}.items(),
        ####                                  columns=["Allele", "Number of Dif Peptides"])
        
        #comp_dict_num_bind.plot(x="Allele", y="Number of Dif Peptides", kind="bar")
        return freq_epitopes
        #return comp_dict_num_bind
        #return comp_dict

    def dist_peptides(self, struct):
        C = defaultdict(int) # store peptide frequency for rank < 0.5
        D = defaultdict(int) # store number of peptides in each allele
        
        for k, v in struct.items():
          peps = v[v["Rank"] < 0.5]
          D[k] = len(peps)
          for pos, pep in zip(peps["Pos"], peps["Peptide"]):
            C[(pos, pep)] += 1

        # only get peptides that occur atleast more than twice
        C = {k:v for k, v in C.items() if v > 2}
        C = dict(sorted(C.items(), key=lambda item: item[0]))

        C = pd.DataFrame(C.items(), columns=["Position and Peptide", "Count"])
        C.plot(x="Position and Peptide", y="Count", kind="bar")
        plt.title("Count of High Rank Peptides for alleles - Frequency > 2")

        D = pd.DataFrame(D.items(), columns=["Allele", "Count"])
        D.plot(x="Allele", y="Count", kind="bar")
        plt.title("Count of Significant Peptide(rank < 0.5) across HLA-A alleles")

    def call_bar(self):
        self.dist_peptides(self.struct1)
        self.dist_peptides(self.struct2)

In [9]:
#Allele E is null, b/w ba1 and ba2

#f1= "_Omicron_BA2_HLA_E.csv"
#f2= "_Omicron_BA1_HLA_E.csv"

f2= "_Omicron_BA2_HLA_C.csv"
f1= "_Omicron_BA1_HLA_C.csv"
#f2= "_Omicron_BA2_HLA_B.csv"
#f1= "_Omicron_BA1_HLA_B.csv"
#f2= "_Omicron_BA2_HLA_A.csv"
#f1= "_Omicron_BA1_HLA_A.csv"

#ca = NetMHC_Comparative_Analysis(f1, f2)
ca = NetMHC_Comparative_Analysis(f1, f2)
pep = [y[1] for x, y in enumerate(ca.compare_top_binders().keys())]
#df = pd.DataFrame(ca.compare_top_binders().items())

OMICRONba1 and OMICRONba2 -- Whatever we say 

In [ ]:
pep#reverse Allele C   

['FASIEKSNI',
 'KVSGNYNYL',
 'VLYNLAPFF',
 'YFPLRSYSF',
 'IRGDEVRQI',
 'TRTQLPPAY',
 'YNLAPFFTF',
 'CYFPLRSYSF',
 'FSNVTWFHV',
 'FLDHKNNKS',
 'YSKHTPIIV',
 'FSRLDKVEAEV',
 'IFSRLDKVEAEV',
 'FSRLDKVEAEVQ',
 'DIFSRLDKVEAEV',
 'FSRLDKVEAEVQI',
 'NDIFSRLDKVEAEV',
 'FSRLDKVEAEVQID',
 'RSYSFRPTY',
 'YSFRPTYGV',
 'LPPAYTNSF',
 'ASIEKSNII']

In [10]:
#UPDATED

pep# Allele C

['FASIEKSNI',
 'KVSGNYNYL',
 'VLYNLAPFF',
 'YFPLRSYSF',
 'IRGDEVRQI',
 'TRTQLPPAY',
 'YNLAPFFTF',
 'CYFPLRSYSF',
 'FSNVTWFHV',
 'FLDHKNNKS',
 'YSKHTPIIV',
 'FSRLDKVEAEV',
 'IFSRLDKVEAEV',
 'FSRLDKVEAEVQ',
 'DIFSRLDKVEAEV',
 'FSRLDKVEAEVQI',
 'NDIFSRLDKVEAEV',
 'FSRLDKVEAEVQID',
 'RSYSFRPTY',
 'YSFRPTYGV',
 'LPPAYTNSF',
 'ASIEKSNII']

In [8]:
#UPDATED

pep#Allele B

['TPINLGRDL',
 'FPLRSYGF',
 'NLITRTQSY',
 'YNFAPFFAF',
 'FFAFKCYGV',
 'YFPLRSYGF',
 'FPLRSYGFR',
 'YGFRPTYGV',
 'RTQSYTNSF',
 'SVLYNFAPF',
 'VNLITRTQSY',
 'YSVLYNFAPF',
 'SVLYNFAPFF',
 'AQKFNGLTVL',
 'VLYNFAPFF',
 'SKVGGNYNY',
 'RSYGFRPTY',
 'QKFNGLTVL',
 'FHAIHVSGT',
 'IHVSGTNGT',
 'YHKNNKSWM',
 'DPFLDVYY',
 'LRSYGFRPTY',
 'LRSYGFRPT',
 'APFFAFKCY',
 'FAPFFAFKCY',
 'FPLRSYGFRPTY',
 'YHKNNKSWMESEF',
 'YYHKNNKSWMESEF',
 'YHKNNKSWMESEFR',
 'SNVTWFHAI']

In [ ]:
pep#Allele A

['CNDPFLDVY',
 'FCNDPFLDVY',
 'CNDPFLDVYY',
 'SYTNSFTRGVY',
 'QSYTNSFTRGVY',
 'SYTNSFTRGVYY',
 'FQFCNDPFLDVY',
 'TQSYTNSFTRGVY',
 'QSYTNSFTRGVYY',
 'SYTNSFTRGVYYP',
 'EFQFCNDPFLDVY',
 'RTQSYTNSFTRGVY',
 'TQSYTNSFTRGVYY',
 'QSYTNSFTRGVYYP',
 'SYTNSFTRGVYYPD',
 'CEFQFCNDPFLDVY',
 'KLDSKVGGNYNYLY',
 'VLYNFAPFFA',
 'VLNDILSRL',
 'SVLNDILSRL',
 'FLPFFSNVTWFHAI',
 'LLPLVSSQCVNLI',
 'FLPFFSNVTWFHA',
 'VLLPLVSSQCVNLI',
 'LLPLVSSQCVNLIT',
 'LFLPFFSNVTWFHA',
 'RDLPQGFSALEPLV',
 'VTWFHAIHV',
 'FFAFKCYGV',
 'FLDVYYHKN',
 'GVYFASTEK',
 'RSYGFRPTY',
 'LDVYYHKNNK',
 'VLYNFAPFFAFK',
 'SVLYNFAPFFAFK',
 'VLYNFAPFFAFKC',
 'YSVLYNFAPFFAFK',
 'SVLYNFAPFFAFKC',
 'VLYNFAPFFAFKCY',
 'SVLNDILSR',
 'DYSVLYNF',
 'YYHKNNKSW',
 'ADYSVLYNF',
 'SVLYNFAPF',
 'YNFAPFFAF',
 'VYYHKNNKSW',
 'VADYSVLYNF',
 'YSVLYNFAPF',
 'LYNFAPFFAF',
 'CYFPLRSYGF',
 'VLYNFAPFFAF',
 'SVLYNFAPFFAF',
 'YSVLYNFAPFFAF',
 'DYSVLYNFAPFFAF',
 'LYNFAPFF',
 'YFPLRSYGF',
 'IYSKHTPINL',
 'YFPLRSYGFR',
 'LYNFAPFFAFK',
 'NCYFPLRSYGF',
 'CYFPLRSYGFR',


In [14]:
a = pd.Series([0, 1, 6, 9])
b = pd.Series([1, 3])
#~(a.isin(b))
~(a.isin(b))

0     True
1    False
2     True
3     True
dtype: bool

In [ ]:
ca.struct1['HLA-E0101']["Rank"] = ca.struct1['HLA-E0101']["Rank"].astype(float)

0
Pos             object
Peptide         object
ID              object
nM              object
Rank           float64
Core            object
H_Avg_Ranks     object
N_binders       object
dtype: object

In [ ]:
ca.struct1['HLA-E0101'].dtypes

0
Pos             object
Peptide         object
ID              object
nM              object
Rank           float64
Core            object
H_Avg_Ranks     object
N_binders       object
dtype: object

In [ ]:
pd.read_csv("_Omicron_BA1_HLA_E.csv").head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-E0101,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UFO69279_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UFO69279_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UFO69279_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UFO69279_1_surf,41043.7,36,FLVLL_PLV,36,0


In [ ]:
pred = pd.read_csv("_Omicron_BA1_HLA_E.csv")#("_Omicron_BA1_HLA_E.csv")   HLA epitopes are binding sites for mAb
hla_names = pred.columns[pred.columns.str.startswith("HLA")].tolist()
hla_names

['HLA-E0101']

In [ ]:
pred[1:][:].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-E0101,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1,0,MFVFLVLL,UFO69279_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UFO69279_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UFO69279_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UFO69279_1_surf,41043.7,36,FLVLL_PLV,36,0
5,4,LVLLPLVS,UFO69279_1_surf,45159.9,80,_LVLLPLVS,80,0


In [ ]:
p = pred[1:][:]
p.columns = pred.iloc[0].tolist()

In [ ]:
pred.iloc[0].tolist()

['Pos', 'Peptide', 'ID', 'nM', 'Rank', 'Core', 'H_Avg_Ranks', 'N_binders']

In [ ]:
p.columns

Index(['Pos', 'Peptide', 'ID', 'nM', 'Rank', 'Core', 'H_Avg_Ranks',
       'N_binders'],
      dtype='object')

In [ ]:
p.head()

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UFO69279_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UFO69279_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UFO69279_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UFO69279_1_surf,41043.7,36,FLVLL_PLV,36,0
5,4,LVLLPLVS,UFO69279_1_surf,45159.9,80,_LVLLPLVS,80,0


In [ ]:
pred.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-E0101,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UFO69279_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UFO69279_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UFO69279_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UFO69279_1_surf,41043.7,36,FLVLL_PLV,36,0


In [ ]:
hla_dty1 = {}
hla_dty2 = {}

In [ ]:
#for i, col_index in enumerate(range(0, len(pred.columns), 3)):
#  #print("i: ", i, " : ", "col_index: ", col_index)
#  hla_dty2[hla_names[i]] = pd.concat([pos_id,
#                                      pred.iloc[:, col_index:col_index+3],
#                                      n_binders], axis=1)

In [ ]:
#df1 = list(hla_dty1.values())[0]
#df1.head()

In [ ]:
df2 = list(hla_dty2.values())[0]
df2.head()

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UJE45220_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UJE45220_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UJE45220_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UJE45220_1_surf,41043.7,36,FLVLL_PLV,36,0
5,4,LVLLPLVS,UJE45220_1_surf,45159.9,80,_LVLLPLVS,80,0


In [ ]:
#hla_df1 = list(hla_dict1.values())[0].reset_index().drop("index", axis=1)
#hla_df1.shape
#hla_dct1 = df1.to_dict()

In [ ]:
#hla_df2 = list(hla_dict2.values())[0].reset_index().drop("index", axis=1)
#hla_dct2 = df2.to_dict()

In [ ]:
#hla_dct2

In [ ]:
#freq_diff_epitopes

{}

In [ ]:
#hla_dct1[k][hla_dct1[k]["Rank"] < 90]

In [ ]:
f1= "_Omicron_BA2_HLA_E.csv"
f2= "_Omicron_BA1_HLA_E.csv"

f3= "_Omicron_BA2_HLA_C.csv"
f4= "_Omicron_BA1_HLA_C.csv"
f5= "_Omicron_BA2_HLA_B.csv"
f6= "_Omicron_BA1_HLA_B.csv"
f7= "_Omicron_BA2_HLA_A.csv"
f8= "_Omicron_BA1_HLA_A.csv"

In [ ]:
df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)
df3 = pd.read_csv(f3)
df4 = pd.read_csv(f4)
df5 = pd.read_csv(f5)
df6 = pd.read_csv(f6)
df7 = pd.read_csv(f7)
df8 = pd.read_csv(f7)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,3,4,6,7,9,10,12,13,15,16,18,19,21,22,24,25,27,28,30,31,33,34,36,37,39,40,42,43,45,46,48,49,51,52,54,55,57,58,60,61,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df8.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-A0101,Unnamed: 4,Unnamed: 5,HLA-A0201,Unnamed: 7,Unnamed: 8,HLA-A0202,...,Unnamed: 55,Unnamed: 56,HLA-A2501,Unnamed: 58,Unnamed: 59,HLA-A2601,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,Pos,Peptide,ID,nM,Rank,Core,nM,Rank,Core,nM,...,Rank,Core,nM,Rank,Core,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UJE45220_1_surf,20865.5,11,_MFVFLVLL,769.3,3.5,_MFVFLVLL,410,...,5,MF_VFLVLL,38500.1,55,MFVFLVL_L,28709.2,28,MFVFLV_LL,7.137,0
2,1,FVFLVLLP,UJE45220_1_surf,24629.7,18,FVFLVLLP_,4569.8,9,FVFLVLLP_,7058.8,...,42,_FVFLVLLP,26598.7,11,FVFLVLLP_,24409.3,17,FVFLVLLP_,18.538,0
3,2,VFLVLLPL,UJE45220_1_surf,25674.6,21,_VFLVLLPL,2779.9,7,_VFLVLLPL,3632.8,...,4.5,VFL_VLLPL,27213.2,11,_VFLVLLPL,26791.6,23,_VFLVLLPL,11.157,0
4,3,FLVLLPLV,UJE45220_1_surf,24348.6,17,FL_VLLPLV,28.2,0.4,FL_VLLPLV,148.1,...,15,_FLVLLPLV,35702.3,35,FLVLLPL_V,26668.1,22,FLV_LLPLV,0.524,9


In [ ]:
df1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-E0101,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UJE45220_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UJE45220_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UJE45220_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UJE45220_1_surf,41043.7,36,FLVLL_PLV,36,0


In [ ]:
df1.columns[3]

'HLA-E0101'

In [ ]:
df3.columns[3]

'HLA-C0303'

In [ ]:
p = df1[1:][:]
p.columns = df1.iloc[0].tolist()

In [ ]:
p.head()

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UJE45220_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UJE45220_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UJE45220_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UJE45220_1_surf,41043.7,36,FLVLL_PLV,36,0
5,4,LVLLPLVS,UJE45220_1_surf,45159.9,80,_LVLLPLVS,80,0


In [ ]:
p.columns

Index(['Pos', 'Peptide', 'ID', 'nM', 'Rank', 'Core', 'H_Avg_Ranks',
       'N_binders'],
      dtype='object')

In [ ]:
p.dtypes

Pos            object
Peptide        object
ID             object
nM             object
Rank           object
Core           object
H_Avg_Ranks    object
N_binders      object
dtype: object

In [ ]:
q = df2[1:][:]
q.columns = df2.iloc[0].tolist()

In [ ]:
q.head()

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UFO69279_1_surf,32283.2,9.5,_MFVFLVLL,9.5,0
2,1,FVFLVLLP,UFO69279_1_surf,45636.3,85,FVF_LVLLP,85,0
3,2,VFLVLLPL,UFO69279_1_surf,36874.2,18,VFLVLLP_L,18,0
4,3,FLVLLPLV,UFO69279_1_surf,41043.7,36,FLVLL_PLV,36,0
5,4,LVLLPLVS,UFO69279_1_surf,45159.9,80,_LVLLPLVS,80,0


In [ ]:
hla_dct1 = p.to_dict()
hla_dct2 = q.to_dict()

In [ ]:
#struct1_top = hla_dct1[k][hla_dct1[k]["Rank"] < 0.5]
#struct2_top = hla_dct2[k][hla_dct2[k]["Rank"] < 0.5]            
#dif = struct1_top[~(struct1_top["Peptide"].isin(struct2_top["Peptide"]))]

In [ ]:
hla_dct1.keys()

dict_keys(['Pos', 'Peptide', 'ID', 'nM', 'Rank', 'Core', 'H_Avg_Ranks', 'N_binders'])

In [ ]:
def typeChange(df):
  ranks = [r for r in df.columns if r.startswith("Rank")]
  for r in ranks:
    df[r] = df[r].astype(float)
  nMs = [n for n in df.columns if n.startswith("nM")]
  for n in nMs:
    df[n] = df[n].astype(float)
  haRanks = [h for h in df.columns if h.startswith("H_Avg_Ranks")]
  for h in haRanks:
    df[h] = df[h].astype(float)
  nBinders = [b for b in df.columns if b.startswith("N_binders")]
  for b in nBinders:
    df[b] = df[b].astype(int)
  return df

In [ ]:
q["Rank"] = q["Rank"].astype(float)
q["nM"] = q["nM"].astype(float)
q["H_Avg_Ranks"] = q["H_Avg_Ranks"].astype(float)
q["N_binders"] = q["N_binders"].astype(int)

In [ ]:
p["Rank"] = p["Rank"].astype(float)
p["nM"] = p["nM"].astype(float)
p["H_Avg_Ranks"] = p["H_Avg_Ranks"].astype(float)
p["N_binders"] = p["N_binders"].astype(int)

In [ ]:
p.dtypes

Pos             object
Peptide         object
ID              object
nM             float64
Rank           float64
Core            object
H_Avg_Ranks    float64
N_binders        int64
dtype: object

In [ ]:
p[p["Rank"] < 0.5].head()

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders
266,265,YLQPRTFL,UJE45220_1_surf,4912.9,0.09,YLQPRTF_L,0.09,1
267,266,LQPRTFLL,UJE45220_1_surf,1201.0,0.01,LQP_RTFLL,0.01,1
503,502,QPYRVVVL,UJE45220_1_surf,8212.0,0.25,_QPYRVVVL,0.25,1
1529,265,YLQPRTFLL,UJE45220_1_surf,470.7,0.01,YLQPRTFLL,0.01,1
1765,501,HQPYRVVVL,UJE45220_1_surf,2711.8,0.03,HQPYRVVVL,0.03,1


In [ ]:
len(p.columns)

8

In [ ]:
struct1_top = p[p["Rank"] < 0.5]
struct2_top = q[q["Rank"] < 0.5]            
dif = struct1_top[~(struct1_top["Peptide"].isin(struct2_top["Peptide"]))]

In [ ]:
dif #f1, f2 are the same

,Pos,Peptide,ID,nM,Rank,Core,H_Avg_Ranks,N_binders


In [ ]:
def colsChange(df):
  cols=pd.Series(df.columns)
  for dup in df.columns[df.columns.duplicated(keep=False)]: 
    cols[df.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 else dup 
                                     for d_idx in range(df.columns.get_loc(dup).sum())]
                                    )
  #df.columns=cols
  return cols

In [ ]:
df3.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,HLA-C0303,Unnamed: 4,Unnamed: 5,HLA-C0401,Unnamed: 7,Unnamed: 8,HLA-C0501,...,Unnamed: 25,Unnamed: 26,HLA-C1402,Unnamed: 28,Unnamed: 29,HLA-C1502,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,Pos,Peptide,ID,nM,Rank,Core,nM,Rank,Core,nM,...,Rank,Core,nM,Rank,Core,nM,Rank,Core,H_Avg_Ranks,N_binders
1,0,MFVFLVLL,UJE45220_1_surf,44918.7,48,M_FVFLVLL,25910.4,18,MFVFL_VLL,46095.4,...,19,M_FVFLVLL,28961,19,MFVFLVL_L,41263.7,43,M_FVFLVLL,20.846,0
2,1,FVFLVLLP,UJE45220_1_surf,45554.9,55,FVFLVLLP_,36514.9,40,FVFLV_LLP,45062.3,...,18,FVFLVLLP_,39881.5,39,FVFLVLLP_,40867.8,42,FVFLVLLP_,29.973,0
3,2,VFLVLLPL,UJE45220_1_surf,41187.4,32,_VFLVLLPL,30101.8,25,_VFLVLLPL,47244.1,...,24,_VFLVLLPL,23859.9,14,VFL_VLLPL,32533.6,22,_VFLVLLPL,26.84,0
4,3,FLVLLPLV,UJE45220_1_surf,45124.7,49,FLVLLPL_V,35445.2,37,_FLVLLPLV,40740.2,...,20,FLVLLPL_V,42928.7,55,FLVLLPL_V,36311.6,29,FLVLLPL_V,21.373,0


In [ ]:
df3.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'HLA-C0303', 'Unnamed: 4',
       'Unnamed: 5', 'HLA-C0401', 'Unnamed: 7', 'Unnamed: 8', 'HLA-C0501',
       'Unnamed: 10', 'Unnamed: 11', 'HLA-C0602', 'Unnamed: 13', 'Unnamed: 14',
       'HLA-C0701', 'Unnamed: 16', 'Unnamed: 17', 'HLA-C0702', 'Unnamed: 19',
       'Unnamed: 20', 'HLA-C0802', 'Unnamed: 22', 'Unnamed: 23', 'HLA-C1203',
       'Unnamed: 25', 'Unnamed: 26', 'HLA-C1402', 'Unnamed: 28', 'Unnamed: 29',
       'HLA-C1502', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34'],
      dtype='object')

In [ ]:
p.columns = colsChange(p)

In [ ]:
p.columns

Index(['Pos', 'Peptide', 'ID', 'nM', 'Rank', 'Core', 'H_Avg_Ranks',
       'N_binders'],
      dtype='object')

In [ ]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]
pepList = []

In [ ]:
alleleList = []
dfsRev = dfs.reverse()
while len(dfs) != 0:
  pp = dfs.pop()
  qq = dfs.pop()
  alleleList.append(pp.columns[3])
  #alleleList.append(qq.columns[3])
  pi = pp[1:][:]
  pi.columns = pp.iloc[0].tolist()
  qi = qq[1:][:]
  qi.columns = qq.iloc[0].tolist()
  if(len(pi.columns) > 8):
    pi.columns = colsChange(pi)
    qi.columns = colsChange(qi)
    pi = typeChange(pi)
    print(pi.dtypes)
    qi = typeChange(qi)
  struct1_top = pi[pi["Rank"] < 0.5]
  struct2_top = qi[qi["Rank"] < 0.5]            
  dif = struct1_top[~(struct1_top["Peptide"].isin(struct2_top["Peptide"]))]
  pepList.append(dif)

TypeError: ignored